# Вывод основываясь на предпочтениях

In [1]:
# Используется обученная модель, 

In [3]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    # Находим все видео с данным тегом
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        # Выбор до 2 видео с этим тегом
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    # Если файл пуст, просто используем финальные video_id
    existing_video_ids = set()  # Пустое множество, так как ничего нет в security.parquet
else:
    existing_video_ids = set(security_df['video_id'])

# Генерация новых video_id в случае совпадений
while any(video_id in existing_video_ids for video_id in final_video_ids):
    # Ищем альтернативные video_id, пока все не будут уникальными
    additional_videos = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
    n=len(final_video_ids), random_state=random.randint(0, 100))
    
    final_video_ids = additional_videos['video_id'].tolist()
    final_video_ids = list(set(final_video_ids))[:10]  # Убедимся, что оставим только 10

# Получаем данные о финальных видео
final_videos = unique_videos[unique_videos['video_id'].isin(final_video_ids)]

# Объединяем с дополнительным DataFrame для получения title и v_pub_datetime
final_videos_info = pd.merge(final_videos, additional_df[['video_id', 'title', 'v_pub_datetime']],
                              on='video_id', how='left')

# Сохранение итоговых video_id в файл security.parquet
final_video_ids_df = pd.DataFrame(final_video_ids, columns=['video_id'])

# Добавляем новые video_id в security.parquet, если они уникальны
new_ids_to_add = final_video_ids_df[~final_video_ids_df['video_id'].isin(existing_video_ids)]

if not new_ids_to_add.empty:
    # Объединяем с существующими данными
    updated_security_df = pd.concat([security_df, new_ids_to_add], ignore_index=True)
    updated_security_df.to_parquet('security.parquet', index=False)

# Вывод финальных видео
print(final_videos_info[['video_id', 'title', 'v_pub_datetime', 'category_id']])

                               video_id  \
0  2231d493-6837-4f09-a766-b847867bc3d6   
1  048a1077-edc2-4f09-bdb9-08c8f461523f   
2  4e972100-6a11-4dc2-9146-bb955526af46   
3  957005c2-a259-45e1-9685-74589887960d   
4  1c1e1973-f492-4361-ad63-4942d2b6fd68   
5  6355965d-ae67-419a-b8c8-d088d1eaecdb   
6  e5fa6850-a969-4fc9-ae57-8f542b8fd196   
7  fd95ff16-da0f-491f-ba74-bee6dc7bb31e   
8  20618a8d-3bd9-4a30-8b32-eabf3681b7fb   
9  b3181c61-2d66-4114-b0ad-e891c08e20c5   

                                               title  \
0            Ирина Хакамада показала свой автомобиль   
1                 Друзья / Friends – 5 сезон 2 серия   
2  мы вызвали сиреноголового в лесу / переписка Д...   
3  «ДНК»: «Отец без документов» | Выпуск от 1 фев...   
4  Человек-паук: Возвращение домой | Spider-Man: ...   
5                    Красивые девушки в нижнем белье   
6                                                  1   
7  Женский стендап: Оля Малащенко - Агрессивный ф...   
8                     

In [12]:
#вывод уже использованных
import pandas as pd

# Чтение Parquet файла
df = pd.read_parquet('security.parquet')

# Вывод первых 5 строк
print(df.head(100))

                                video_id
0   9f74dc31-255f-448c-8f52-4d0c3c0100da
1   7523cdb0-e050-46f3-a8b8-b51454e9ed59
2   0119fbc7-db81-4ef1-99d5-55600348d9d0
3   53d7fd98-ba32-4913-86a9-11ee1f01948f
4   6230fb26-948b-45cc-8df7-526c644db075
..                                   ...
85  b3181c61-2d66-4114-b0ad-e891c08e20c5
86  20618a8d-3bd9-4a30-8b32-eabf3681b7fb
87  6355965d-ae67-419a-b8c8-d088d1eaecdb
88  e5fa6850-a969-4fc9-ae57-8f542b8fd196
89  4e972100-6a11-4dc2-9146-bb955526af46

[90 rows x 1 columns]


In [11]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    existing_video_ids = set()  # Пустое множество, так как ничего нет в security.parquet
else:
    existing_video_ids = set(security_df['video_id'])

# Генерация новых video_id в случае совпадений
while any(video_id in existing_video_ids for video_id in final_video_ids):
    final_video_ids = []
    
    for tag in top_tags['tag']:
        videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
        
        if not videos_with_tag.empty:
            selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
            final_video_ids.extend(selected_videos['video_id'].tolist())
            remaining_slots = 10 - len(final_video_ids)

    if remaining_slots > 0:
        popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
            n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
                  random_state=random.randint(0, 100))

        final_video_ids.extend(popular_others['video_id'].tolist())

    final_video_ids = list(set(final_video_ids))  # Удаляем дубликаты
    final_video_ids = final_video_ids[:10]  # Оставляем только 10 видео

# Создание итогового DataFrame с информацией о видео
final_videos_info = unique_videos[unique_videos['video_id'].isin(final_video_ids)].copy()

# Переименование столбца video_id в uid
final_videos_info.rename(columns={'video_id': 'uid'}, inplace=True)

# Сохранение итогового DataFrame в файл JSON
final_videos_info.to_json('final_videos_info.json', orient='records', lines=True)

print("Итоговая таблица сохранена в final_videos_info.json.")

# Загрузка DataFrame из JSON для проверки
df = pd.read_json('final_videos_info.json', lines=True)

# Вывод первых 100 строк
print(df.head(100))

Итоговая таблица сохранена в final_videos_info.json.
                                    uid   category_id  v_likes  v_dislikes  \
0  33231c5e-8ade-4361-b488-c8575287a221       Сериалы       94           1   
1  99dc8571-3206-4686-85e8-fe1ea6da4436   Развлечения       23           0   
2  e50f5c7a-a4e2-419b-8a13-18ea219b599a  Телепередачи       19           0   
3  3d2895f8-d928-4971-81af-74adaf2288b4   Мультфильмы      668          28   
4  30e52a56-9d48-498b-ac68-617868a59b29   Развлечения      179           4   
5  a9672629-8ea1-4349-8091-e1dfc0c411d4    Аудиокниги        8           0   
6  f1d6389b-9a11-4bcf-adaf-8aa2ada24c61         Детям       91           3   
7  58aac622-479c-4011-ae6d-1001b52a9a50   Мультфильмы        6           0   
8  73ed63d5-42ce-4eed-a7bd-5fb2eb37a752   Путешествия        2           0   
9  3f96ca41-00b6-450d-9aae-23d9c13f628b   Развлечения        0           0   

   v_frac_avg_watchtime_1_day_duration  v_frac_avg_watchtime_7_day_duration  \
0        

In [ ]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    # Находим все видео с данным тегом
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        # Выбор до 2 видео с этим тегом
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Добавление топ-3 видео по регионам
top_regions = regions_df.nlargest(3, 'region').video_id.tolist()
final_video_ids.extend(top_regions)

# Добавление топ-3 видео по городам
top_cities = city_df.nlargest(3, 'city').video_id.tolist()
final_video_ids.extend(top_cities)

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    # Если файл пуст, просто используем финальные video_id

## С учётом региона и города

In [ ]:
import pandas as pd
import joblib
import random
import time

# Загрузка модели
model = joblib.load('random_forest_model.joblib')

# Загрузка данных из файлов
filtred_df = pd.read_parquet('tags-filtered.parquet')
logs_df = pd.read_parquet('logsV02.parquet')
tags_df = pd.read_parquet('users/user0_tags.parquet')
regions_df = pd.read_parquet('users/user0_regions.parquet')
city_df = pd.read_parquet('users/user0_city.parquet')
additional_df = pd.read_parquet('parquet-filtered/filtred.parquet')

# Объединение данных по video_id
merged_df = pd.merge(filtred_df, logs_df, on='video_id', how='inner')

# Ограничиваемся первыми 1000 записями
merged_df = merged_df.head(1000)

# Признаки
features = ['category_id', 'v_likes', 'v_dislikes',
            'v_frac_avg_watchtime_1_day_duration',
            'v_frac_avg_watchtime_7_day_duration',
            'v_frac_avg_watchtime_30_day_duration']

# Преобразование категориальных переменных для новых данных
X_new = pd.get_dummies(merged_df[features], drop_first=True)

# Обработка данных: замена бесконечных значений и NaN
X_new.replace([float('inf'), float('-inf')], pd.NA, inplace=True)
X_new.dropna(inplace=True)

# Сохранение индексов оставшихся строк
valid_indices = X_new.index

# Обеспечение соответствия признаков модели
X_new = X_new.reindex(columns=model.feature_names_in_, fill_value=0)

# Выполнение предсказаний
predictions = model.predict(X_new)

# Создание нового столбца с предсказанными значениями только для действительных индексов
merged_df.loc[valid_indices, 'predicted_popularity'] = predictions

# Удаление дубликатов по video_id
unique_videos = merged_df.drop_duplicates(subset='video_id')

# Сортировка по предсказанной популярности
unique_videos = unique_videos.sort_values(by='predicted_popularity', ascending=False)

# Поиск максимальных 3 значений percent с тегом
top_tags = tags_df.nlargest(3, 'percent')

# Список для хранения финальных видео
final_video_ids = []

# Выбор случайного числа для изменения выборки
random.seed(time.time())

# Поиск двух популярных видео для каждого из топовых тегов
for tag in top_tags['tag']:
    # Находим все видео с данным тегом
    videos_with_tag = unique_videos[unique_videos['category_id'].str.contains(tag, na=False)]
    
    if not videos_with_tag.empty:
        # Выбор до 2 видео с этим тегом
        selected_videos = videos_with_tag.sample(n=min(2, len(videos_with_tag)), random_state=random.randint(0, 100))
        final_video_ids.extend(selected_videos['video_id'].tolist())

# Добавление топ-3 видео по регионам
top_regions = regions_df.nlargest(3, 'region').video_id.tolist()
final_video_ids.extend(top_regions)

# Добавление топ-3 видео по городам
top_cities = city_df.nlargest(3, 'city').video_id.tolist()
final_video_ids.extend(top_cities)

# Если всё ещё не достигли 10 видео, добавляем случайные видео из других категорий
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    popular_others = unique_videos[~unique_videos['video_id'].isin(final_video_ids)].sample(
        n=min(remaining_slots, len(unique_videos[~unique_videos['video_id'].isin(final_video_ids)])),
              random_state=random.randint(0, 100))
        
    final_video_ids.extend(popular_others['video_id'].tolist())

# Удаляем дубликаты, если они есть
final_video_ids = list(set(final_video_ids))

# Получаем финальные 10 видео
final_video_ids = final_video_ids[:10]

# Проверка на наличие файла security.parquet и его содержимого
try:
    security_df = pd.read_parquet('security.parquet')
except ValueError:
    # Если файл не существует или пуст, то создаем пустая DataFrame
    security_df = pd.DataFrame(columns=['video_id'])

if security_df.empty:
    # Если файл пуст, просто используем финальные video_id
    pass
else:
    # Если файл не пустой, проверяем наличие video_id из финального списка
    existing_video_ids = security_df['video_id'].unique().tolist()
    
    # Фильтруем финальные video_id, исключая уже существующие
    final_video_ids = [vid for vid in final_video_ids if vid not in existing_video_ids]

# Проверяем, есть ли еще места до 10 видео
remaining_slots = 10 - len(final_video_ids)

if remaining_slots > 0:
    # Получаем дополнительные видео из уникальных видео, которые ещё не в финальном списке
    additional_videos = unique_videos[~unique_videos['video_id'].isin(final_video_ids)]
    
    if not additional_videos.empty:
        # Выбираем случайные видео для заполнения оставшихся слотов
        more_videos = additional_videos.sample(n=min(remaining_slots, len(additional_videos)),
                                               random_state=random.randint(0, 100))
        final_video_ids.extend(more_videos['video_id'].tolist())

# Удаляем дубликаты и ограничиваем длину списка до 10
final_video_ids = list(set(final_video_ids))[:10]

# Печатаем или сохраняем финальные video_id
print("Финальные рекомендованные video_id:", final_video_ids)

# Здесь можно добавить код для сохранения рекомендаций, например, в новый parquet файл
final_recommendations_df = pd.DataFrame({'video_id': final_video_ids})
final_recommendations_df.to_parquet('final_recommendations.parquet', index=False)